In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
import sys

def next_state(state, action, action_level):
    next_state = state.copy()
    if (action > 2*action_level) or action < 0:
        print("action = ", action)
        sys.exit("error, action undefined! nxt")

    if action >= 1 and action <= action_level:
        # then buy
        
        if state['HandCash'] > 0:
            next_state['HandCash'] = state['HandCash']*(1 - action/action_level)
            next_state['NumStock'] = state['NumStock'] + \
                                     (state['HandCash']/state['StockPrice'])*(action/action_level)

    elif action >= (action_level + 1) and action <= (action_level + action_level) :
        # then sell
        
        if state['NumStock'] > 0:
            next_state['NumStock'] = state['NumStock']*(1 - (action-action_level)/action_level)
            next_state['HandCash'] = state['HandCash'] + \
                                     (state['StockPrice']*state['NumStock'])*((action-action_level)/action_level)
    
    return next_state


def reward(state, init_invest):
    return (state['NumStock']*state['StockPrice'] + state['HandCash']) - init_invest

def my_reward(curr, last):
    return (curr['NumStock']*curr['StockPrice'] + curr['HandCash']) - \
           (last['NumStock']*last['StockPrice'] + last['HandCash'])

In [3]:
def psy(price_list):
    n = 0
    for i in range(1, len(price_list)):
        if price_list[i] > price_list[i-1]:
            n += 1
    return (n/float(len(price_list) - 1))*100


def feature_6_original(price_list,action,action_level):
    psy_num = psy(price_list)
    if(psy_num <= 25): # market oversell we can buy
        if(action <= action_level):
            return (action + action_level + 1)/float(2*action_level + 1)
        elif(action_level < action <= 2*action_level):
            return (action - action_level)/float(2*action_level + 1)
        else:
            return (action_level+1)/float(2*action_level + 1)
    if(25 < psy_num < 75):#hold to observe situation
        if(action <= action_level):
            return 2*(action_level - action + 1)/float(2*action_level + 1)
        elif(action_level < action <= 2*action_level):
            return 2*(2*action_level - action + 1)/float(2*action_level + 1)
        else:
            return 1
    if(psy_num >= 75):#market overbuy we can sell
        if(action <= action_level):
            return (action_level - action + 1)/float(2*action_level + 1)
        elif(action_level < action <= 2*action_level):
            return (action + 1)/float(2*action_level + 1)
        else:
            return (action_level+1)/float(2*action_level + 1)
        
        
def feature_6_dis(price_list, action, action_level):
    psy_num = psy(price_list)
    if(psy_num <= 25): # market oversell we can buy
        if(action <= action_level):
            return (action + action_level)/float(2*action_level)
        elif(action_level < action <= 2*action_level):
            return (2*action_level - action)/float(2*action_level)
        
    if(25 < psy_num < 75):#hold to observe situation
        if(action <= action_level):
            return (action_level - action)/float(action_level)
        elif(action_level < action <= 2*action_level):
            return (2*action_level - action)/float(action_level)
        
    if(psy_num >= 75):#market overbuy we can sell
        if(action <= action_level):
            return (action_level - action)/float(2*action_level)
        elif(action_level < action <= 2*action_level):
            return (action)/float(2*action_level)
        
def feature_6_conti(price_list, action, action_level):
    psy_num = psy(price_list)
    spsy_num = psy_num/100

    if(action <= action_level):
        saction = (action_level - action)/(2*action_level) # scaled action
    else:
        saction = action/(2*action_level)
            
    return 1-abs(spsy_num-saction)

# same as feature_6_dis here
def feature_6_all(price_list, action, action_level):
    psy_num = psy(price_list)
    if(psy_num <= 25): # market oversell we can buy
        if(action == action_level):
            return 1
        else:
            return 0
        
    if(25 < psy_num < 75):#hold to observe situation
        if(action == 0):
            return 1
        else:
            return 0
        
    if(psy_num >= 75):#market overbuy we can sell
        if(action == 2*action_level):
            return 1
        else:
            return 0

In [4]:
from random import randint

def V_value(price_list, action_level):
    v = 0
    first = 1 # flag of the frist iteration
    v_act = ''
    for action in range(0, (2*action_level + 1)):
        
        # choose feature_5_conti or feature_5_all here!!
        curr_q = feature_6_all(price_list, action, action_level)
        
        if first == 1:
            first = 0
            v = curr_q
            v_act = action
        elif curr_q > v:
            v = curr_q
            v_act = action
        elif curr_q == v:
            flip = randint(0, 1)
            if flip == 1:
                v = curr_q
                v_act = action
        

    return [v, v_act]

##### lists of data in each folder: copy and paste into the cell below
* ../input/  
inputFolderFormat = "../input/{0}"
inputCsvList = ['QCOM.csv', 'GOOG.csv', 'NOK.csv', '^DJI.csv', '^TWII.csv', 
           '^N225.csv', '^GDAXI.csv', '^IXIC.csv', '^KS11.csv', '^SSEC.csv', 
           'NSU.DE_10_5_5.csv', 'DAI.DE_10_5_5.csv', 'BMW.DE_10_5_5.csv', 
           '0050.TW.csv', '1216.TW.csv', '1301.TW.csv', '2002.TW.csv', 
           '2330.TW.csv', '2409.TW.csv', '2454.TW.csv', '1704.TW.csv']

* ../input/2001_2010/  
inputFolderFormat = "../input/2001_2010/{0}"
inputCsvList = ['QCOM.csv', 'NOK.csv', '^DJI.csv', '^TWII.csv', 
            '^N225.csv', '^GDAXI.csv', '^IXIC.csv', '^KS11.csv', 
            '^SSEC.csv', 'NSU.DE.csv', 'DAI.DE.csv', 'BMW.DE.csv']  
* ../input/fall/  
inputFolderFormat = "../input/fall/{0}"  
inputCsvList = ['1314.TW.csv', '2323.TW.csv', '2349.TW.csv', 
'2353.TW.csv', '2405.TW.csv', '2409.TW.csv', '2601.TW.csv', 
'2885.TW.csv', '2888.TW.csv', '6116.TW.csv']

In [5]:
from random import randint
import math
import stockstats
from os import listdir

inputFolderFormat = "../input/temp/{0}"
inputCsvList = listdir('../input/temp')


psyallrwdlist = list()

action_level = 1
last_rwd_func = reward
avg_point_rwd_func = my_reward

init_invest = 10000

data_result = list()

for resource_data in inputCsvList:

    input_csv = inputFolderFormat.format(resource_data)
    qcom = pd.read_csv(input_csv)
    
    qcom.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
    qcom = qcom.drop(index=0).reset_index(drop=True)
    qcom = qcom.drop(index=0).reset_index(drop=True)
    qcom = qcom.iloc[::-1].reset_index(drop=True)
    qcom.to_csv('file_namePSY.csv', sep=',', index=False)
    qcom = pd.read_csv('file_namePSY.csv')
    
    qclose = qcom['Close']
    qclose = qclose.fillna(method='ffill')
    qclose = qclose.fillna(method='bfill')
    qclose = qclose.tolist()
    np.isnan(qclose).any()
    
    psyrwdlist = list()
    
    fx_num = 12
    
    # initial state
    state = {'NumStock': 0, 'StockPrice': qclose[0], 'HandCash': init_invest}

    for point in range(fx_num, len(qclose)):

        # new price revealed!
        state['StockPrice'] = qclose[point]

        # prepare for the arguments passed later
        price_list = qclose[point-fx_num+1:point+1]
        
        # determine the next state
        act = (V_value(price_list, action_level))[1]
                
        state = next_state(state, act, action_level)
        
        psyrwdlist.append(state['HandCash'] + state['StockPrice']*state['NumStock'] - 10000)
    psyallrwdlist.append(psyrwdlist)
    #print(resource_data, last_rwd_func(state, init_invest))
    
    data_result.append(state['HandCash'] + state['StockPrice']*state['NumStock'] - 10000)
print(data_result)

[-5110.893889016682, 24803.899834143005, -2296.899802740393, 10785.295431041217, -6799.829440016513, 9230.76923076923, -5950.736053719009, 523.7167303583574, 6930.464522891274, -629.1450949469727, -9625.341493572803, 17205.797908437595, 267.8571428571431, 140000.0, -636.5326544634536, 6825.396825396823, -4780.582524271845, 1278.2351560007337, -2537.0466501479723, -1146.7535383131417, 9166.340483517979, 37678.04903477151, 13790.697674418607, 5613.792424477477, 22582.34069675832, 649.1687008399349, -3621.359247807318, -4363.636363636364, -3555.324333812334, 3588.763486748945, 409.63228792843256, 10689.201040840064, 113664.12213740457, -288.461538461539, 15780.783637152279, 22490.018745373218, -3088.235294117647, 7592.502156601342, -3929.032258064516, 11171.37654354602, 18121.751198741626, 31288.998574381432, 6850.450444532911, 4255.864435848616, -1616.7277259349103, 17202.73782247111, 3827.1611895545684, -7307.499603618318, -9249.942244012629, 3047.5536301612156, 2314.2818914863383, 1652

In [6]:
for i in range(0, len(inputCsvList)):
    input_csv = inputFolderFormat.format(inputCsvList[i])
    data = pd.read_csv(input_csv)
    close = data['Close']
    close = close.fillna(method='ffill')
    close = close.tolist()
    plt.title(inputCsvList[i])
    plt.plot(close)
    plt.show()
    plt.plot(psyallrwdlist[i])
    plt.show()

KeyError: 'Close'